In [9]:
import os
import json
import shutil
import iaflow
import importlib
import pickle as pkl
import tensorflow as tf

importlib.reload(iaflow)
IAFlow = iaflow.IAFlow

In [10]:
def custom_builder(input_shape):
  model = tf.keras.models.Sequential([
    tf.keras.layers.Input(input_shape),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
  ])
  return model

params_notifier = { # Parameters for notifier, see documentation https://pypi.org/project/notify-function/#description
  'title': 'Training update',
  'webhook_url': os.environ.get('WEBHOOK_URL'),
  'frequency_epoch': 20 # This will send a notification every 20 epochs, by default it is every epoch
}

all_data = tf.data.Dataset.from_tensor_slices((
  tf.random.uniform([1000, 2]),
  tf.random.uniform([1000, 1])
))
train_ds = all_data.take(5)
val_ds = all_data.skip(5)

In [11]:
# Creation of the IAFlow instance
shutil.rmtree('./models', ignore_errors=True)
ia_maker = IAFlow(
  models_folder='./models',
  params_notifier=params_notifier,
  builder_function=custom_builder
)

In [12]:
ia_maker.add_dataset(
  name='dataset_1',
  epochs=10,
  batch_size=32,
  shuffle_buffer=512,
  train_ds=train_ds,
  val_ds=val_ds
)

Dataset dataset_1 added


True

In [13]:
model_1_data = ia_maker.add_model(
  model_name='model_1',
  model_params={ 'input_shape': (2, 1) }, # Parameters for builder function
  load_model_params={}, # Parameters for model loading, see documentation of tf.keras.models.load_model
  compile_params={ # Parameters for model compilation, see documentation tf.keras.models.compile
    'optimizer': 'adam', 'loss': 'mse',
    'metrics': ['accuracy']
  },
)
#print(f'Data of model_1: {json.dumps(model_1_data, indent=4)}')

GPU info: /physical_device:GPU:0


Model model_1/2022.07.18_10.04.03 added


In [14]:
print('Show all models:')
ia_maker.show_models()

Show all models:
Models:
  model_1
    2022.07.18_10.04.03
      model_ident: model_1_(2, 1)
      model_params: {'input_shape': (2, 1)}
      compile_params: {'optimizer': 'adam', 'loss': 'mse', 'metrics': ['accuracy']}
      load_model_params: {'filepath': './models/model_1/2022.07.18_10.04.03/model_1_(2, 1)_checkpoint.h5'}
      check_path: ./models/model_1/2022.07.18_10.04.03/model_1_(2, 1)_checkpoint.h5
      path_model: ./models/model_1/2022.07.18_10.04.03
      log_dir: ./models/model_1/2022.07.18_10.04.03/logs


In [17]:
ia_maker.train(
  model_1_data,
  epochs=5,
  dataset_name='dataset_1'
)

Force creation is False. Deleting old logs and model
Creating model
Epoch 1/5


2022-07-18 10:04:58.540624: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-07-18 10:04:58.549085: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - ETA: 0s - loss: 0.1005 - accuracy: 0.0000e+00

2022-07-18 10:04:58.969940: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 1s/step - loss: 0.1005 - accuracy: 0.0000e+00 - val_loss: 0.0842 - val_accuracy: 0.0000e+00
Epoch 2/5
1/1 [==============================] - 0s 352ms/step - loss: 0.1004 - accuracy: 0.0000e+00 - val_loss: 0.0843 - val_accuracy: 0.0000e+00
Epoch 3/5
1/1 [==============================] - 0s 364ms/step - loss: 0.1002 - accuracy: 0.0000e+00 - val_loss: 0.0844 - val_accuracy: 0.0000e+00
Epoch 4/5
1/1 [==============================] - 0s 316ms/step - loss: 0.1001 - accuracy: 0.0000e+00 - val_loss: 0.0844 - val_accuracy: 0.0000e+00
Epoch 5/5
1/1 [==============================] - 1s 551ms/step - loss: 0.1000 - accuracy: 0.0000e+00 - val_loss: 0.0844 - val_accuracy: 0.0000e+00
Training time: 2.994709014892578


In [18]:
ia_maker.clear_session()

print('Loading model trained')
ia_maker.train(
  model_1_data,
  epochs=5,
  dataset_name='dataset_1'
)

Loading model trained
Loading model from ./models/model_1/2022.07.18_10.04.03/model_1_(2, 1)_checkpoint.h5
Epoch 1/5


2022-07-18 10:05:22.334058: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - ETA: 0s - loss: 0.0998 - accuracy: 0.0000e+00

2022-07-18 10:05:22.783857: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 1s/step - loss: 0.0998 - accuracy: 0.0000e+00 - val_loss: 0.0843 - val_accuracy: 0.0000e+00
Epoch 2/5
1/1 [==============================] - 0s 425ms/step - loss: 0.0996 - accuracy: 0.0000e+00 - val_loss: 0.0844 - val_accuracy: 0.0000e+00
Epoch 3/5
1/1 [==============================] - 0s 353ms/step - loss: 0.0995 - accuracy: 0.0000e+00 - val_loss: 0.0845 - val_accuracy: 0.0000e+00
Epoch 4/5
1/1 [==============================] - 0s 498ms/step - loss: 0.0993 - accuracy: 0.0000e+00 - val_loss: 0.0846 - val_accuracy: 0.0000e+00
Epoch 5/5
1/1 [==============================] - 0s 422ms/step - loss: 0.0992 - accuracy: 0.0000e+00 - val_loss: 0.0847 - val_accuracy: 0.0000e+00
Training time: 2.79247784614563


In [19]:
ia_maker.delete_model(model_1_data, delete_folder=True)

Model model_1/2022.07.18_10.04.03 deleted


True